In [1]:
import pandas as pd

In [2]:
all_notes = ['(',')','~','*','+','->']
operaters = ['~','*','+','->']

In [3]:
def f_not(a):
    return int(not(a))

def f_or(a,b):
    return int(a==1 or b==1)

def f_and(a,b):
    return int(a==1 and b==1)

def f_condition(a,b):
    return int(not(a==1 and b==0))

def calcu(note_list,para_list):
    note = note_list.pop()
    a = para_list.pop()
    if note == '~':
        para_list.append(f_not(a))
        return
    b = para_list.pop()
    if note == '*':
        para_list.append(f_and(b,a))
    elif note == '+':
        para_list.append(f_or(b,a))
    elif note == '->':
        para_list.append(f_condition(b,a))

In [4]:
def calculate_formula(formula_str, para_value_map):
    note_tree = []
    para_stack = []
    note_point = note_tree
    outer_note_tree = note_tree
    note_prior = operaters
    i = 0
    while i < len(formula_str):
        s = formula_str[i]
        # 处理特殊符号（或、与）
        if s == '\\' and formula_str[i+1] == '/':
            s = '+'
            i += 1
        elif s == '/' and formula_str[i+1] == '\\':
            s = '*'
            i += 1
        elif s == '-' and formula_str[i+1] == '>':
            s = '->'
            i += 1
        # 处理括号
        if s == '(':
            inter_note_tree = []
            note_point.append(inter_note_tree)
            outer_note_tree = note_point
            note_point = inter_note_tree
        elif s ==')':
            while len(note_point)!=0:
                calcu(note_point,para_stack)
                # print(' ','\t',para_stack,'\t',note_tree)
            note_point = outer_note_tree
            note_point.pop()
        elif s in note_prior:
            if len(note_point) == 0:
                note_point.append(s)
                i += 1
                continue
            if note_prior.index(s) < note_prior.index(note_prior[-1]):
                note_point.append(s)
            else:
                calcu(note_point,para_stack)
                note_point.append(s)
        else:
            para_stack.append(para_value_map[s])
        i += 1
        # print(s,'\t',para_stack,'\t',note_tree)
    # print(note_point,len(note_point))
    while len(note_point)!=0:
        calcu(note_point,para_stack)
        # print(' ','\t',para_stack,'\t',note_tree)
    return para_stack.pop()

In [5]:
def extract_para(formula):
    notes = all_notes
    para_list = []
    for s in formula:
        if s not in notes and s not in para_list and s not in ['\\','/','-','>']:
            para_list.append(s)
    return para_list

def extract_child_formula(formula):
    pre_probe = []
    post_probe = []
    child_formulas = []
    for i in range(len(formula)):
        if formula[i] == '(':
            pre_probe.append(i)
        elif formula[i] == ')':
            post_probe.append(i)
    while len(pre_probe) != 0:
        child_formulas.append(formula[pre_probe.pop(0):post_probe.pop(0)+1])
    child_formulas.append(formula)
    return child_formulas

def create_true_value(paras):
    times = 2**len(paras)
    true_values = []
    for i in range(times):
        this_time_values = []
        b = f'{i:0{len(paras)}b}'
        for s in b:
            this_time_values.append(int(s))
        true_values.append(this_time_values)
    return true_values

In [6]:
def calcu_with_all_true_value(paras,child_formulas):
    true_values = create_true_value(paras)
    ans_values = []
    for t_v in true_values:
        pvm = dict(zip(paras,t_v))
        this_time_values = []
        for i in range(len(child_formulas)):
            c_f = child_formulas[i]
            this_time_result = calculate_formula(c_f,pvm)
            this_time_values.append(this_time_result)
        t_v.extend(this_time_values)
        ans_values.append(t_v)
    return ans_values

In [8]:
def draw_true_value_table(formula):
    paras = extract_para(formula)
    child_formulas = extract_child_formula(formula)
    ans = calcu_with_all_true_value(paras,child_formulas)
    column_name = paras
    column_name.extend(child_formulas)
    result_table = pd.DataFrame(ans,columns=column_name)
    print(result_table)

In [9]:
formula = '(A/\B)->C'
draw_true_value_table(formula)

   A  B  C  (A/\B)  (A/\B)->C
0  0  0  0       0          1
1  0  0  1       0          1
2  0  1  0       0          1
3  0  1  1       0          1
4  1  0  0       0          1
5  1  0  1       0          1
6  1  1  0       1          0
7  1  1  1       1          1
